In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from datetime import datetime

In [2]:
DATAOS_FQDN = "dataos-training.dataos.app"

token = "Y2xvdWRjb3N0LjU3ODViZDAzLWM3ZjMtNGQ5Zi1hMTJjLWQ1ZjA4NTgwYjJkYw=="

In [ ]:
azure = pd.read_excel("DataPoint_AzureCost_JulyToSept2024_LastOne.xlsx")
snowflake = pd.read_excel("SF_CostResults_JulyToSept2024_LatestOne.xlsx")
gcp = pd.read_csv("simulated_gcp_billing_data.csv")

In [ ]:
# === Normalize schemas ===
azure_df = azure.rename(columns={
    'Region': 'region',
    'Allocation': 'allocation',
    'SubscriptionName': 'subscription',
    'UsageDate': 'usage_date',
    'Cost': 'cost'
})[['region', 'allocation', 'subscription', 'usage_date', 'cost']]
azure_df['platform'] = 'Azure'
azure_df['usage_date'] = pd.to_datetime(azure_df['usage_date'], unit='D', origin='1899-12-30', errors='coerce')

In [ ]:
snowflake_df = snowflake.rename(columns={
    'REGION': 'region',
    'ALLOCATION': 'allocation',
    'ACCOUNT_NAME': 'subscription',
    'START_TIME': 'usage_date',
    'AMOUNT': 'cost'
})[['region', 'allocation', 'subscription', 'usage_date', 'cost']]
snowflake_df['platform'] = 'Snowflake'
snowflake_df['usage_date'] = pd.to_datetime(snowflake_df['usage_date'], errors='coerce')


In [ ]:
gcp_df = gcp.rename(columns={
    'region': 'region',
    'allocation': 'allocation',
    'project_name': 'subscription',
    'usage_start_time': 'usage_date',
    'final_cost': 'cost'
})[['region', 'allocation', 'subscription', 'usage_date', 'cost']]
gcp_df['platform'] = 'GCP'
gcp_df['usage_date'] = pd.to_datetime(gcp_df['usage_date'], errors='coerce')
gcp_df = gcp_df.loc[:, ~gcp_df.columns.duplicated()]

In [ ]:

# === Merge all ===
df = pd.concat([azure_df, snowflake_df, gcp_df], ignore_index=True)

In [ ]:

# === Preprocess ===
df = df.dropna(subset=['usage_date', 'cost'])
df['cost'] = pd.to_numeric(df['cost'], errors='coerce')
df = df[df['cost'] > 0]
df['month'] = df['usage_date'].dt.to_period('M').astype(str)

In [ ]:

# === Summary Metrics ===
summary = df.groupby('platform')['cost'].sum().reset_index().rename(columns={'cost': 'total_cost'})
monthly_trend = df.groupby(['month', 'platform'])['cost'].sum().reset_index()

In [ ]:
# === Forecasting ===
def forecast_cost(grouped_df, forecast_months=1):
    result = {}
    for name, group in grouped_df:
        group = group.groupby('month')['cost'].sum().reset_index()
        group['month_index'] = np.arange(len(group))
        model = LinearRegression()
        model.fit(group[['month_index']], group['cost'])
        next_index = len(group)
        prediction = model.predict([[next_index]])[0]
        result[name] = round(prediction, 2)
    return result

In [ ]:
# Example: Forecast by Platform
platform_forecast = forecast_cost(df.groupby('platform'))

In [ ]:
# === Plotting (Optional) ===
def plot_trend(df, group_col='platform'):
    pivot_df = df.pivot(index='month', columns=group_col, values='cost')
    pivot_df.plot(title=f'Monthly Cost Trend by {group_col.capitalize()}', figsize=(10, 5))
    plt.xlabel('Month')
    plt.ylabel('Cost')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot_trend(monthly_trend, group_col='platform')  # Optional plot

In [ ]:
# === Print Outputs ===
print("=== Summary Cost by Platform ===")
print(summary)
print("\n=== Forecasted Cost by Platform ===")
print(platform_forecast)

In [ ]:
# === Convert forecast dictionary to DataFrame ===
forecast_df = pd.DataFrame.from_dict(platform_forecast, orient='index', columns=['forecasted_cost']).reset_index()
forecast_df = forecast_df.rename(columns={'index': 'platform'})

# === Save to CSV or view ===
forecast_df.to_csv("forecasted_cost_by_platform.csv", index=False)
print("\n=== Forecasted Cost DataFrame ===")
print(forecast_df)


In [ ]:
save(name="dataos://lakehouse:siva/device_events_process", dataframe=final_df, format="iceberg", mode="overwrite")

In [ ]:
final_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .save("lakehouse.siva.device_events_process")